In [1]:
import numpy as np
import torch
from torch import nn
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

import data_load
import data_process
import model
import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = data_load.load_turbine_standardized_data_without_time(
    "data/Turbine_Data_Penmanshiel_11_2021-01-01_-_2021-07-01_1051.csv")
data_imputer = KNNImputer(n_neighbors=10)
data = data_process.impute_data(data)
data = data_process.normalize_data(data)
data = np.array(data, dtype='float32')

AttributeError: module 'data_load' has no attribute 'load_turbine_standardized_data_without_time'

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

epochs = 3000
k = 1
tscv = TimeSeriesSplit(n_splits=2, test_size=int(np.size(data, axis=0)*0.1))
i=0

for i, (train_index, test_index) in enumerate(tscv.split(data)):
    if i == 0:
        continue
    ae_model = model.RnnAutoEncoder(input_size=np.size(data, axis=1)).to(device)
    loss = nn.MSELoss()
    # optimizer = torch.optim.SGD(ae_model.parameters(), lr=0.001)
    optimizer = torch.optim.Adam(ae_model.parameters(), lr=0.001)
    X_train, X_test = data[train_index], data[test_index]
    for epoch in range(epochs):
        print(f"\n=====epoch {epoch}/{epochs}=====")
        train_model.train(data_x=X_train, data_y=X_train, model=ae_model, loss_fn=loss, optimizer=optimizer, batch_size=np.size(data, axis=0))

    res = train_model.autoencoder_test(X_test, X_test, ae_model, 1000)
    # break

In [ ]:
import scipy

res_sorted = np.sort(res)
res_cdf = 1. * np.arange(len(res)) / (len(res) - 1)
# plt.hist(res, bins=200)
plt.plot(res_sorted, res_cdf)
plt.xlim(0, 150)
plt.show()